In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import time
from os import listdir
from os.path import join


from utils_era import extract_era5_chunk

In [2]:
gdf = gpd.read_file('../data_v02/gadm41_USA_1.json')
states = ['Minnesota', 'Iowa', 'Wisconsin', 'NorthDakota', 'SouthDakota', 'Nebraska', 'Illinois', 'Indiana', 'Missouri', 'Michigan', 'Kansas']
gdf = gdf[gdf.NAME_1.isin(states)].reset_index(drop=True)
bbox = np.round(gdf.union_all().bounds)

In [3]:
step = 0.1#0.0045
lons = np.arange(bbox[0], bbox[2], step)
lats = np.arange(bbox[1], bbox[3], step)

lon_grid, lat_grid = np.meshgrid(lons, lats)

In [23]:
def process_era5_data(grid, batch_size=500):
    all_features = []
    all_dates = []
    
    for i in range(0, len(grid), batch_size):
        chunk = grid.iloc[i:i+batch_size]

        try:
            t = time.time()
            chunk_features, chunk_dates = extract_era5_chunk(chunk)
    
            all_features.append(chunk_features)
            all_dates.extend(chunk_dates)

            print(f'Processed {i + len(chunk_features)}/{len(grid)} locations | Time: {round(time.time() - t, 3)}')
        
        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
    df = pd.concat(all_features, axis=0).reset_index(drop=True)
    df = pd.concat([df, grid], axis=1)
    
    return df

In [28]:
dates = pd.date_range('2024-01-01', '2024-01-31', freq='1D')
dfs = []
for date in dates:
    mesh_df = pd.DataFrame({
    'lon': lon_grid.ravel(),
    'lat': lat_grid.ravel(),
    'date': [date]*len(lat_grid.ravel())
    })
    
    df = process_era5_data(mesh_df, batch_size=150)

    dfs.append(df)

Processed 150/28600 locations | Time: 12.976
Processed 300/28600 locations | Time: 17.456
Processed 450/28600 locations | Time: 6.509
Processed 600/28600 locations | Time: 11.675
Processed 750/28600 locations | Time: 30.254
Processed 900/28600 locations | Time: 28.386
Processed 1050/28600 locations | Time: 32.832
Processed 1200/28600 locations | Time: 29.066
Processed 1350/28600 locations | Time: 27.537
Processed 1500/28600 locations | Time: 30.741
Processed 1650/28600 locations | Time: 29.834
Processed 1800/28600 locations | Time: 32.014
Processed 1950/28600 locations | Time: 34.171
Processed 2100/28600 locations | Time: 36.624
Processed 2250/28600 locations | Time: 30.449
Processed 2400/28600 locations | Time: 22.661
Processed 2550/28600 locations | Time: 28.249
Processed 2700/28600 locations | Time: 26.554
Processed 2850/28600 locations | Time: 27.479
Processed 3000/28600 locations | Time: 29.611
Processed 3150/28600 locations | Time: 35.151
Processed 3300/28600 locations | Time: 33

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [29]:
pd.concat(dfs)

ValueError: No objects to concatenate